In [1]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm_notebook
import tqdm
import seaborn as sns
from choices import param_dict
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import  IterativeImputer 
from sklearn.impute import SimpleImputer , KNNImputer
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingClassifier
sns.set(font_scale=1.3, rc={'axes.facecolor':'white', 'figure.facecolor':'white' , 'axes.grid':True} , style="whitegrid")
# 
%load_ext autoreload
%autoreload 2
import time 
tstart = time.time()
feat_to_drop = param_dict['hardness']+param_dict['IRAC']

In [2]:

from utilities import deets
from choices import get_train_data , param_dict
classes = ['AGN' ,'STAR' , 'YSO' ,  'CV' , 'LMXB' , 'HMXB' ,'ULX','PULSAR']
flag = {
    'conf_flag' : 0 , 
    'streak_src_flag' : 0 , 
    'extent_flag' : 0 , 
    'pileup_flag' : 0 , 
    }
ret_dict= {
    'clf': False,
 'prob_table': True,
 'acc': True,
 'pr_score': True,
 'precision': True,
 'recall': True , 
 'roc_auc_score' : False
 }



In [3]:

file = f'../compiled_data_v3/imputed_data_v2/x_phot_minmax_modeimp.csv'
data = get_train_data(flags = flag, classes= classes , offset = 1, file=file)
data = data.drop(columns = feat_to_drop)
deets(data,0)


_____________________________________________________
------------------------------
Number of Objects : 7703
Number of Columns : 42
_____________________________________________________


In [ ]:
import joblib
from tqdm import tqdm
from utilities import cv , get_score
from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier

# {'max_depth': 30, 'n_estimators': 400} for MODE imputation
# {'max_depth': 180, 'n_estimators': 400} for RF imputation
for i in (range(5,15)):
    print('-------------------------------------')
    print(f'DOING {i}th Iteration')
    rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
    res_final  = cv(
        {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
        {'model' : rf , 'name' :'GB'} , ## CHANGE HERE########################################################
        k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = False)
    score = get_score([res_final['prob_table']])
    joblib.dump(score , f'result/classifier_scores/rf_mode_{i}.pkl')
    joblib.dump(res_final , f'result/prob_table/rf_mode_{i}.pkl' , compress=3)

In [7]:
get_score([res_final['prob_table']] , sc_average= 'macro')

{'classes': array(['AGN', 'CV', 'HMXB', 'LMXB', 'PULSAR', 'STAR', 'ULX', 'YSO'],
       dtype=object),
 'num_src':         pred_class
 STAR          2743
 AGN           2307
 YSO           1155
 HMXB           789
 ULX            253
 CV             184
 LMXB           138
 PULSAR         134,
 'avg_scores': {'balanced_accuracy': 0.7680378457790499,
  'accuracy': 0.902116058678437,
  'precision': 0.7365595425181941,
  'recall': 0.7680378457790499,
  'f1': 0.7502748453724473},
 'mcc': 0.8685322596571063,
 'class_scores':         recall_score  precision_score  f1_score
 class                                          
 AGN         0.932777         0.968357  0.950234
 CV          0.542169         0.489130  0.514286
 HMXB        0.842246         0.798479  0.819779
 LMXB        0.811189         0.840580  0.825623
 PULSAR      0.504950         0.380597  0.434043
 STAR        0.943011         0.959169  0.951021
 ULX         0.644550         0.537549  0.586207
 YSO         0.923412         0.91

In [5]:
import joblib
from tqdm import tqdm
from utilities import cv , get_score
from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier

# {'max_depth': 30, 'n_estimators': 400} for MODE imputation
# {'max_depth': 180, 'n_estimators': 400} for RF imputation
for i in (range(5,15)):
    print('-------------------------------------')
    print(f'DOING {i}th Iteration')
    # rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
    gb = GradientBoostingClassifier(random_state=np.random.randint(0,999999))
    res_final  = cv(
        {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
        {'model' : gb, 'name' :'GB'} , ## CHANGE HERE########################################################
        k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = True)
    score = get_score([res_final['prob_table']])
    joblib.dump(score , f'result/classifier_scores/gb_mode_{i}.pkl')
    joblib.dump(res_final, f'result/prob_table/gb_mode_{i}.pkl' , compress=3)

-------------------------------------
DOING 5th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 6th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 7th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 8th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 9th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in resp

In [18]:
def score_mean(file , num=5):
    labels = ['AGN' ,'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
    scores = ['precision' , 'recall' , 'f1_score']
    all_feat = {
        'AGN' : {
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } ,
        'YSO' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'STAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'HMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'LMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } , 
        'ULX' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'CV':{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'PULSAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        }
    }
    for l in labels:
        for i in range(num):
            model = joblib.load(f'result/classifier_scores/{file}_{i}.pkl')
            all_feat[l]['precision'].append(model['class_scores']['precision_score'][l])
            all_feat[l]['recall'].append(model['class_scores']['recall_score'][l])
            all_feat[l]['f1_score'].append(model['class_scores']['f1_score'][l])
    all_mean = {}
    for l in labels:
        all_mean[l] = {}
        for s in scores:
            all_mean[l][s] = np.mean(all_feat[l][s])*100
    all_std = {}
    for l in labels:
        all_std[l] = {}
        for s in scores:
            all_std[l][s] = np.std(all_feat[l][s])*100
    mean_df = pd.DataFrame(all_mean)
    std_df = pd.DataFrame(all_std)
    return mean_df , std_df

In [28]:
import joblib
score = score_mean('gb_mode' , 5)
display(score[0].T)
display(score[1].T)

,precision,recall,f1_score
AGN,97.900976,89.578288,93.554365
STAR,97.173790,91.440860,94.220133
YSO,91.323651,93.803307,92.546856
HMXB,83.152012,87.566845,85.299527
LMXB,78.607862,81.398601,79.933531
ULX,47.091859,75.545024,58.013787
CV,42.215190,59.759036,49.456699
PULSAR,28.991428,56.237624,38.253909


,precision,recall,f1_score
AGN,0.073536,0.416366,0.225975
STAR,0.102457,0.168725,0.118724
YSO,0.231568,0.185850,0.207429
HMXB,0.984614,0.638360,0.685951
LMXB,3.677993,0.948578,1.994356
ULX,1.008256,0.709319,0.918763
CV,0.930888,1.228679,0.260772
PULSAR,0.888895,1.584158,1.047590


In [26]:
import joblib
score = score_mean('gb_mode' , 15)
display(score[0].T)
display(score[1].T)

,precision,recall,f1_score
AGN,97.839297,89.606124,93.541487
STAR,97.102954,91.376344,94.152494
YSO,91.187763,93.652451,92.403281
HMXB,83.384978,87.397504,85.340758
LMXB,77.465074,80.792541,79.065962
ULX,47.094498,75.039494,57.864335
CV,42.381689,60.080321,49.692752
PULSAR,28.346037,55.775578,37.582852


,precision,recall,f1_score
AGN,0.133853,0.336962,0.184300
STAR,0.152102,0.198831,0.131253
YSO,0.283979,0.276075,0.205416
HMXB,0.770292,0.661221,0.486182
LMXB,2.772887,0.879625,1.491830
ULX,1.113081,1.089933,1.051159
CV,1.022605,1.608432,1.048964
PULSAR,1.160929,1.831609,1.365245


In [72]:
score[1]

,AGN,STAR,YSO,HMXB,LMXB,ULX,CV,PULSAR
precision,0.153113,0.156210,0.210555,0.315574,0.448078,0.889318,1.970650,2.410174
recall,0.159759,0.017559,0.170548,0.181346,0.342586,0.975889,0.851936,1.154644
f1_score,0.073306,0.080487,0.088599,0.189966,0.300596,0.856258,1.065672,1.421311


# Average scores

In [121]:
score = {
    'accuracy' : [],
    'precision' : [], 
    'recall' : [],
    'f1_score' : [] , 
    'mcc' : []
}
for i in range(5):
    temp = joblib.load(f'result/prob_table/rf_mode_{i}.pkl')
    score['accuracy'].append(temp['avg_scores']['accuracy']*100)
    score['precision'].append(temp['avg_scores']['precision']*100)
    score['recall'].append(temp['avg_scores']['recall']*100)
    score['f1_score'].append(temp['avg_scores']['f1']*100)
    score['mcc'].append(temp['mcc'])

In [122]:
print('accuracy :' , np.mean(score['accuracy']) , np.std(score['accuracy']))
print('precision :' , np.mean(score['precision']) , np.std(score['precision']))
print('recall :' , np.mean(score['recall']) , np.std(score['recall']))
print('f1 :' , np.mean(score['f1_score']) , np.std(score['f1_score']))
print('mcc :' , np.mean(score['mcc']) , np.std(score['mcc']))


accuracy : 90.92820978839413 0.05408974855511051
precision : 90.32620716634699 0.06435587305543901
recall : 90.92820978839413 0.05408974855511051
f1 : 90.41222662077027 0.05854608335311202
mcc : 0.8764900029262218 0.0007293516152392633


In [124]:
prob_table = joblib.load('result/classifier_scores/rf_mode_0.pkl')
prob_table

{'classes': array(['AGN', 'CV', 'HMXB', 'LMXB', 'PULSAR', 'STAR', 'ULX', 'YSO'],
       dtype=object),
 'num_src':         pred_class
 STAR          2829
 AGN           2467
 YSO           1207
 HMXB           787
 ULX            151
 LMXB           109
 CV             101
 PULSAR          52,
 'avg_scores': {'balanced_accuracy': 0.6989552185297148,
  'accuracy': 0.9086070362196547,
  'precision': 0.9022987594200812,
  'recall': 0.9086070362196547,
  'f1': 0.9033687333540134},
 'mcc': 0.8755667280539116,
 'class_scores':         recall_score  precision_score  f1_score
 class                                          
 AGN         0.967850         0.939603  0.953517
 CV          0.403614         0.663366  0.501873
 HMXB        0.852941         0.810673  0.831270
 LMXB        0.755245         0.990826  0.857143
 PULSAR      0.227723         0.442308  0.300654
 STAR        0.952688         0.939555  0.946076
 ULX         0.488152         0.682119  0.569061
 YSO         0.943429         0.8

In [134]:
ret_dict= {
    'clf': True,
 'prob_table': True,
 'acc': True,
 'pr_score': True,
 'precision': True,
 'recall': True , 
 'roc_auc_score' : True
 }
rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
res_final  = cv(
    {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
    {'model' : rf , 'name' :'GB'} , ## CHANGE HERE########################################################
    k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = False)
score = get_score([res_final['prob_table']])
joblib.dump(res_final , 'result/all_result/rf.pkl')

Doing 20 fold cross-validation


100%|██████████| 20/20 [02:21<00:00,  7.10s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
>>> Training the final classifier RandomForestClassifier(max_depth=30, n_estimators=400, n_jobs=-1,
                       random_state=997096)
[DONE] Classifier is trained | acces via ['clf'] in the response


['result/all_result/rf.pkl']

In [133]:
ret_dict= {
    'clf': True,
 'prob_table': True,
 'acc': True,
 'pr_score': True,
 'precision': True,
 'recall': True , 
 'roc_auc_score' : True
 }
rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
gb = GradientBoostingClassifier()
res_final  = cv(
    {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
    {'model' : gb , 'name' :'GB'} , ## CHANGE HERE########################################################
    k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = False)
score = get_score([res_final['prob_table']])
joblib.dump(res_final , 'result/all_result/gb.pkl')

Doing 20 fold cross-validation


100%|██████████| 20/20 [20:42<00:00, 62.11s/it]


[RESULT] Overall Accuracy : 0.92
[INFO] Validation probability table is available as ['prob_table'] in response
>>> Training the final classifier GradientBoostingClassifier()
[DONE] Classifier is trained | acces via ['clf'] in the response


['result/all_result/gb.pkl']

# CMP distribution

In [ ]:
rf = joblib.load('result/all_result/gb.pkl')